In [1]:
# dependencies
import pandas as pd
import os
import mimetypes

# path to resource files
path = "Resources"

# get a file name list
files = [f for f in os.listdir(path) if (mimetypes.guess_type(f)[0] == "text/csv" and f.startswith('clean_'))]
print(files)

['clean_hawaii_stations.csv', 'clean_hawaii_measurements.csv']


In [2]:
st = pd.read_csv(os.path.join(path, files[0]), dtype=object)
ms = pd.read_csv(os.path.join(path, files[1]), dtype=object)
# ms

In [3]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
Base = declarative_base()

import pymysql
pymysql.install_as_MySQLdb()

In [4]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [6]:
Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longtitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, name={self.name}, station={self.station}"

class Measurement(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"id={self.id}, station={self.station}"

In [7]:
Base.metadata.create_all(engine)

In [8]:
st_data = st.to_dict(orient='records')
st_data[0]

{'elevation': '3',
 'latitude': '21.2716',
 'longitude': '-157.8168',
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [9]:
ms_data = ms.to_dict(orient='records')
ms_data[0]

{'date': '2010-01-01', 'prcp': '0.08', 'station': 'USC00519397', 'tobs': '65'}

In [10]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [11]:
st_table = sqlalchemy.Table('stations', metadata, autoload=True)
ms_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [12]:
conn.execute(st_table.insert(), st_data)

In [13]:
conn.execute(ms_table.insert(), ms_data)

In [14]:
conn.execute("select * from stations limit 10").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, None, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, None, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, None, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, None, 306.6),
 (6, 'USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, None, 19.5),
 (7, 'USC00519281', 'WAIHEE 837.5, HI US', 21.45167, None, 32.9),
 (8, 'USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, None, 0.9),
 (9, 'USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, None, 152.4),
 (10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0)]

In [15]:
conn.execute("select * from measurements limit 10").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70),
 (6, 'USC00519397', '2010-01-08', 0.0, 64),
 (7, 'USC00519397', '2010-01-09', 0.0, 68),
 (8, 'USC00519397', '2010-01-10', 0.0, 73),
 (9, 'USC00519397', '2010-01-11', 0.01, 64),
 (10, 'USC00519397', '2010-01-12', 0.0, 61)]

In [16]:
# conn.execute("select * from measurements inner join stations on measurements.station=stations.station").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (1, 'USC00519397', '2010-01-01', 0.08, 65, 10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63, 10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74, 10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76, 10, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70, 1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, None, 3.0),
 (5, 'USC00519397', '2010-01-07', 0.06,